# Loading CMSSW into Jupyter

To teach Jupyter about a new environment, we will create a new `kernel`. In Jupyter, kernels are different combinations of languages and libraries. So, for instance, one kernel could be a python kernel with CMSSW_9_1_0 and another would be CMSSW_8_1_0. This exercise uses CMSSW_10_3_1, so we'll make a new kernel called `cmsdas-preexercise`, which links to CMSSW_10_3_1.

To make the kernel, select the cell below and press `Shift+Enter`. Don't worry about the contents of the cell, this is simply code to make a CMSSW checkout, create a wrapper script, then teach Jupyter about that script.

In [ ]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_10_3_1"
KERNEL_NAME="cmsdas-preexercise"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
if [ ! -d cmssw-env ]; then
    scramv1 project --name cmssw-env CMSSW $CMSSW_VER 
fi

# Now, install additional python packages
cd cmssw-env
eval `scramv1 runtime -sh`

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper2.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
pushd $DIR
eval `scramv1 runtime -sh`
popd
exec python2 "$@"
EOF
chmod +x bin/python_wrapper2.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper2.sh", 
  "-m", 
  "IPython.kernel", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"

If you see
```
Loaded CMSSW_10_3_1 into cmsdas-preexercise!
```
in the previous output, then you successfully loaded your kernel! Please select `File -> Close and Halt` from the menu to close this notebook, and proceed to the `exercises` notebook in the file browser